In [2]:
import demucs
model = "htdemucs"
extensions = ["mp3", "wav", "ogg", "flac"] 
two_stems = None
mp3 = True
mp3_rate = 320
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False 

in_path = '/Users/dominicvian/Desktop/Goldsmiths Year 3/Machine Learning/FINAL PROJECT/demucs/'
out_path = '/Users/dominicvian/Desktop/Goldsmiths Year 3/Machine Learning/FINAL PROJECT/demucs_output/'

In [3]:
#@title Useful functions, don't forget to execute
import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO

# def find_files(in_path):
#     out = []
#     for file in Path(in_path).iterdir():
#         if file.suffix.lower().lstrip(".") in extensions:
#             out.append(file)
#     print(f"Found {len(out)} files in {in_path}")
#     return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    files = ['/Users/dominicvian/Desktop/Goldsmiths Year 3/Machine Learning/FINAL PROJECT/demucs/pipe reversions prev.mp3']
    print (files)
    if not files:
        print("NOPE")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")


def from_upload():
    out_path = Path('separated')
    in_path = Path('tmp_in')
    
    if in_path.exists():
        rmtree(in_path)
    in_path.mkdir()
    
    if out_path.exists():
        rmtree(out_path)
    out_path.mkdir()
    
    uploaded = files.upload()
    for name, content in uploaded.items():
        (in_path / name).write_bytes(content)
    separate(in_path, out_path)


In [11]:
separate()

['/Users/dominicvian/Desktop/Goldsmiths Year 3/Machine Learning/FINAL PROJECT/demucs/pipe reversions prev.mp3']
Going to separate the files:
/Users/dominicvian/Desktop/Goldsmiths Year 3/Machine Learning/FINAL PROJECT/demucs/pipe reversions prev.mp3
With command:  python3 -m demucs.separate -o /Users/dominicvian/Desktop/Goldsmiths Year 3/Machine Learning/FINAL PROJECT/demucs_output/ -n htdemucs --mp3 --mp3-bitrate=320
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /Users/dominicvian/Desktop/Goldsmiths Year 3/Machine Learning/FINAL PROJECT/demucs_output/htdemucs
Separating track /Users/dominicvian/Desktop/Goldsmiths Year 3/Machine Learning/FINAL PROJECT/demucs/pipe reversions prev.mp3


100%|████████████████████████████████████████████████████████████████████████| 210.6/210.6 [03:50<00:00,  1.10s/seconds]


In [4]:
# This is manual upload and download :)
from_upload()
!zip -r separated.zip separated
#files.download('./separated.zip')

NameError: name 'files' is not defined

@inproceedings{defossez2021hybrid,
               title = {Hybrid Spectrogram and Waveform Source Separation},
               author = {D{\'e}fossez, Alexandre},
               booktitle = {Proceedings of the ISMIR 2021 Workshop on Music Source Separation},
               year = {2021}
               }